In [5]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from collections import defaultdict

def get_user_ratings(num_users, num_items):
    R = np.zeros((num_users, num_items))
    for user in range(num_users):
        print(f"Enter ratings for User {user + 1}:")
        for item in range(num_items):
            rating = input(f"Rating for Item {item + 1} (0 if not rated): ")
            R[user, item] = float(rating)
    return R

def apply_svd(R, n_components):
    svd = TruncatedSVD(n_components=n_components)
    U = svd.fit_transform(R)
    Sigma = svd.singular_values_
    Vt = svd.components_
    R_pred = np.dot(np.dot(U, np.diag(Sigma)), Vt)
    return R_pred

def calculate_mae_rmse(R, R_pred):
    mask = R != 0  # Only consider non-zero entries
    mae = mean_absolute_error(R[mask], R_pred[mask])
    rmse = np.sqrt(mean_squared_error(R[mask], R_pred[mask]))
    return mae, rmse

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

def f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def hyperparameter_tuning(R):
    pipe = Pipeline([
        ('svd', TruncatedSVD())
    ])
    param_grid = {
        'svd__n_components': [2, 3, 4, 5],
        'svd__tol': [0.0, 0.1, 0.2]
    }
    gs = GridSearchCV(pipe, param_grid, cv=3, scoring='neg_mean_squared_error')
    gs.fit(R)
    best_params = gs.best_params_
    return best_params

def get_user_feedback(R_pred):
    feedback = []
    for user_idx, user_ratings in enumerate(R_pred):
        print(f"\nPredicted ratings for User {user_idx + 1}:")
        for item_idx, rating in enumerate(user_ratings):
            print(f"Item {item_idx + 1}: {rating:.2f}")
            feedback.append((user_idx, item_idx, rating))
    return feedback

def main():
    num_users = int(input("Enter the number of users: "))
    num_items = int(input("Enter the number of items: "))

    R = get_user_ratings(num_users, num_items)
    print("\nUser-Item Ratings Matrix:\n", R)

    best_params = hyperparameter_tuning(R)
    print("\nBest Parameters from Hyperparameter Tuning:", best_params)

    R_pred = apply_svd(R, n_components=best_params['svd__n_components'])
    print("\nPredicted Ratings:\n", R_pred)

    mae, rmse = calculate_mae_rmse(R, R_pred)
    print(f"\nEvaluation Metrics:\nMAE: {mae}, RMSE: {rmse}")

    feedback = get_user_feedback(R_pred)

    # Example prediction format for precision/recall calculation
    predictions = [(uid, iid, R[uid, iid], R_pred[uid, iid], None) for uid in range(num_users) for iid in range(num_items) if R[uid, iid] != 0]

    precisions, recalls = precision_recall_at_k(predictions)
    avg_precision = np.mean(list(precisions.values()))
    avg_recall = np.mean(list(recalls.values()))
    f1 = f1_score(avg_precision, avg_recall)

    print(f"\nPrecision: {avg_precision}, Recall: {avg_recall}, F1 Score: {f1}")

if __name__ == "__main__":
    main()


Enter the number of users: 3
Enter the number of items: 5
Enter ratings for User 1:
Rating for Item 1 (0 if not rated): 1
Rating for Item 2 (0 if not rated): 0
Rating for Item 3 (0 if not rated): 0
Rating for Item 4 (0 if not rated): 2
Rating for Item 5 (0 if not rated): 3
Enter ratings for User 2:
Rating for Item 1 (0 if not rated): 0
Rating for Item 2 (0 if not rated): 3
Rating for Item 3 (0 if not rated): 2
Rating for Item 4 (0 if not rated): 5
Rating for Item 5 (0 if not rated): 1
Enter ratings for User 3:
Rating for Item 1 (0 if not rated): 1
Rating for Item 2 (0 if not rated): 3
Rating for Item 3 (0 if not rated): 4
Rating for Item 4 (0 if not rated): 0
Rating for Item 5 (0 if not rated): 1

User-Item Ratings Matrix:
 [[1. 0. 0. 2. 3.]
 [0. 3. 2. 5. 1.]
 [1. 3. 4. 0. 1.]]

Best Parameters from Hyperparameter Tuning: {'svd__n_components': 2, 'svd__tol': 0.0}

Predicted Ratings:
 [[ 1.2448805   6.06337279  3.15643282 15.69573729  7.08448932]
 [ 4.51004337 21.68857187 18.55107042 30

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/usr/local/lib/python3.10/